# Teoría del sistema M/M/1/K - Kevin Cabreja 10146048

## 1. Descripción del modelo

Un sistema **M/M/1/K** es un modelo de teoría de colas con las siguientes características:

- Las llegadas siguen un proceso **Poisson** con tasa **λ**.
- Los tiempos de servicio son **exponenciales** con tasa **μ**.
- Hay **1 servidor**.
- La capacidad total del sistema es:
  - **1** paquete en servicio  
  - **K** paquetes en espera  
  Por lo tanto:  
  **Capacidad total = K + 1**
- Si el sistema está lleno, los paquetes que llegan se **descartan** (bloqueo).

---

## 2. Factor de carga

El parámetro clave es:

```
rho = λ / μ
```

- Si rho < 1 → sistema estable.  
- Si rho ≈ 1 → congestionado.  
- Si rho > 1 → el sistema tendería a infinito si no tuviera límite K.

---

## 3. Probabilidades estacionarias

La probabilidad de que el sistema tenga **n** paquetes es:

```
πₙ = π₀ * ρⁿ
```

La probabilidad base es:

```
π₀ = (1 - ρ) / (1 - ρ^(K + 2))
```

*(Recordar que la capacidad total es K + 1 estados, de 0 a K + 1 → por eso aparece K + 2 en la fórmula).*

---

## 4. Probabilidad de bloqueo

La probabilidad de que el sistema esté lleno (y por tanto se pierda un paquete) es:

```
P_bloqueo = π_(K + 1)
```

Es decir:

```
P_bloqueo = π₀ * ρ^(K + 1)
```

---

## 5. Utilización del servidor

Es la probabilidad de que el servidor esté ocupado:

```
Utilización = 1 - π₀
```

---

## 6. Longitud promedio del sistema

El número promedio de paquetes dentro del sistema:

```
L = sum( n * πₙ )   para n = 0 … K + 1
```

---

## 7. Longitud promedio del búfer

La cola no incluye el paquete que está siendo atendido:

```
L_buffer = L - Utilización
```

---

## 8. Tasa efectiva de servicio

Debido a que algunos paquetes son bloqueados:

```
λ_efectiva = λ * (1 - P_bloqueo)
```

---

## 9. Tiempo promedio de espera en la cola

Se obtiene usando Little para los paquetes que sí fueron atendidos:

```
Wq = L_buffer / λ_efectiva
```

---

# Resumen de fórmulas

```
ρ = λ / μ
πₙ = π₀ * ρⁿ
π₀ = (1 - ρ) / (1 - ρ^(K + 2))
P_bloqueo = π₀ * ρ^(K + 1)
Utilización = 1 - π₀
L = Σ(n * πₙ)
L_buffer = L - Utilización
λ_efectiva = λ * (1 - P_bloqueo)
Wq = L_buffer / λ_efectiva
```
---
 
# Gráficos:

![Grafica](1.png)
 


![Grafica](2.png)



![Grafica](3.png) 



![Grafica](4.png)

---

# Codigo Python:

```


In [24]:
import random, math
from collections import deque

# ----- SIMULACIÓN M/M/1/K -----
def simulate_mm1k_ctmc(lam, mu, K_buffer, N_arrival_attempts=10**6, seed=1):
    random.seed(seed)
    Ksys = K_buffer + 1
    t = 0.0
    n = 0
    arrivals = 0
    blocked = 0
    served = 0
    buffer_time_area = 0.0
    busy_time = 0.0

    while True:
        rate_arr = lam
        rate_dep = mu if n > 0 else 0.0
        rate_total = rate_arr + rate_dep
        if rate_total <= 0:
            break
        dt = -math.log(random.random()) / rate_total
        buffer_len = max(0, n - 1)
        buffer_time_area += buffer_len * dt
        if n > 0:
            busy_time += dt
        t += dt
        if random.random() < rate_arr / rate_total:
            arrivals += 1
            if n == Ksys:
                blocked += 1
            else:
                n += 1
            if arrivals >= N_arrival_attempts and n == 0:
                break
        else:
            if n > 0:
                served += 1
                n -= 1
            if arrivals >= N_arrival_attempts and n == 0:
                break
    T = t
    lambda_eff = served / T
    L_buffer = buffer_time_area / T
    Wq = L_buffer / lambda_eff
    return {
        "P_block": blocked / arrivals,
        "util": busy_time / T,
        "L_buffer": L_buffer,
        "Wq": Wq,
        "lambda_eff": lambda_eff,
        "arrivals": arrivals,
        "blocked": blocked,
        "served": served
    }

# ----- SIMULACIÓN M/M/2/K -----
def simulate_mm2k_ctmc(lam, mu, K_buffer, N_arrival_attempts=10**6, seed=1):
    random.seed(seed)
    c = 2
    Ksys = K_buffer + c
    t = 0.0
    n = 0
    arrivals = 0
    blocked = 0
    served = 0
    buffer_time_area = 0.0
    busy_time = 0.0

    while True:
        rate_arr = lam
        rate_dep = min(n, c) * mu
        rate_total = rate_arr + rate_dep
        if rate_total <= 0:
            break
        dt = -math.log(random.random()) / rate_total
        buffer_len = max(0, n - c)
        buffer_time_area += buffer_len * dt
        busy_time += min(n, c) * dt
        t += dt
        if random.random() < rate_arr / rate_total:
            arrivals += 1
            if n == Ksys:
                blocked += 1
            else:
                n += 1
            if arrivals >= N_arrival_attempts and n == 0:
                break
        else:
            if n > 0:
                served += 1
                n -= 1
            if arrivals >= N_arrival_attempts and n == 0:
                break
    T = t
    lambda_eff = served / T
    L_buffer = buffer_time_area / T
    Wq = L_buffer / lambda_eff
    return {
        "P_block": blocked / arrivals,
        "util": busy_time / (c * T),
        "L_buffer": L_buffer,
        "Wq": Wq,
        "lambda_eff": lambda_eff,
        "arrivals": arrivals,
        "blocked": blocked,
        "served": served
    }

# ------ EJECUCIÓN ------
lambda_val = 180
K = 5

for mu in [200, 400]:
    print(f"\n=== M/M/1/K, μ={mu} ===")
    sim1 = simulate_mm1k_ctmc(lambda_val, mu, K)
    for k, v in sim1.items():
        print(f"{k}: {v}")

    print(f"\n=== M/M/2/K, μ={mu} ===")
    sim2 = simulate_mm2k_ctmc(lambda_val, mu, K)
    for k, v in sim2.items():
        print(f"{k}: {v}")



=== M/M/1/K, μ=200 ===
P_block: 0.10194726689646276
util: 0.8081586009106068
L_buffer: 1.7749664495961528
Wq: 0.010994986105952669
lambda_eff: 161.4341694015592
arrivals: 1000017
blocked: 101949
served: 898068

=== M/M/2/K, μ=200 ===
P_block: 0.002965
util: 0.4483690937999488
L_buffer: 0.21357583239169053
Wq: 0.001191780224457615
lambda_eff: 179.2073974787507
arrivals: 1000000
blocked: 2965
served: 997035

=== M/M/1/K, μ=400 ===
P_block: 0.00474998575004275
util: 0.4480425718505329
L_buffer: 0.345174894362278
Wq: 0.001928540837657096
lambda_eff: 178.9824138656128
arrivals: 1000003
blocked: 4750
served: 995253

=== M/M/2/K, μ=400 ===
P_block: 3.0999969000031e-05
util: 0.22485630344708266
L_buffer: 0.024129949576163375
Wq: 0.00013425200400870593
lambda_eff: 179.73623376674962
arrivals: 1000001
blocked: 31
served: 999970
